In [54]:
import yfinance as yf
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import math
from numpy import linalg as la
import matplotlib.pyplot as plt
import math
import pandas as pd
import yfinance as yf
import statistics as stat
from scipy.special import ndtri
from scipy.stats import norm
import random
from sklearn.metrics import mean_squared_error

In [55]:
stock_name = "BTC-USD"
data = yf.download(stock_name, start="2023-08-13", end="2023-09-04")
data["Date"] = data.index

[*********************100%***********************]  1 of 1 completed


In [56]:
prices = data.copy()
# Calcule os retornos diários
prices['Returns'] = (prices['Close'] / prices['Close'].shift(1) - 1) * 100

# Calcule a média e o desvio padrão dos retornos diários
mean_daily_return = prices['Returns'].mean()
std_daily_return = prices['Returns'].std()

# Calcule a volatilidade anual
trading_days_per_year = 22  # Assumindo 252 dias úteis em um ano
volatility_annual = std_daily_return * np.sqrt(trading_days_per_year)

In [57]:
volatility_annual

11.656935411774667

In [62]:
mu, sig, N = 1.1, 1, 100000
pts = []


def q(x):
    return (1 / (math.sqrt(2 * math.pi * sig ** 2))) * (math.e ** (-((x - mu) ** 2) / (2 * sig ** 2)))

def MCMC(n):
    r = np.zeros(1)
    p = q(r[0])
    pts = []

    for i in range(N):
        rn = r + np.random.uniform(-1, 1)
        pn = q(rn[0])
        if pn >= p:
            p = pn
            r = rn
        else:
            u = np.random.rand()
            if u < pn / p:
                p = pn
                r = rn
        pts.append(r)

    pts = random.sample(pts, len(pts))
    pts = np.array(pts)
    
    return pts

def MH(stock_name, data):
    hist_data = yf.download(stock_name, start="2023-08-13", end="2023-09-04")
    hist_data = hist_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
    hist_data = hist_data['Close']
    hist_data = np.array(hist_data)
    stock_pred = []
    data = data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
    dt1 = data['Close']
    maturnity = 1
    volatility = 0.12
    risk_free = 0.1
    timestep = 1
    steps = 22
    delta_t = maturnity / steps
    i = 0
    stock_pred.append(dt1[-1])
    while timestep < steps:
        stock_price = stock_pred[-i]
        time_exp = maturnity - delta_t * timestep
        # Generate z_t using MCMC method
        pts = MCMC(N)
        stock_price = stock_price * math.exp(((risk_free - 0.5 * (
            math.pow(volatility, 2))) * delta_t + volatility * math.sqrt(delta_t) * pts[timestep + 5]))
        stock_pred.append(stock_price)
        i = i + 1
        timestep = timestep + 1
    
    rmse = math.sqrt(mean_squared_error(hist_data, stock_pred))
    print(f'RMSE MCMC: {rmse}')
    
    
    # build graphs
    preds_gr = np.reshape(stock_pred, (22,))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=hist_data, mode='lines+markers',  name='historical', marker_color='#39304A'))
    fig.add_trace(go.Scatter(x=list(range(0, 21)), y=preds_gr, mode='lines+markers', name='predictions', marker_color='#FFAA00'))
    fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',  
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),
                  title_text = f'{stock_name} MCMC prediction', title_x = 0.5,
                  xaxis_title="Timestep",
                  yaxis_title="Stock price",
                  margin=dict(l=0, r=0, t=30, b=0))
    fig.show()
    
    return stock_pred, rmse

In [63]:
mcmc_pred, mcmc_rmse = MH(stock_name, data)
mcmc_pred = np.vstack(mcmc_pred)

[*********************100%***********************]  1 of 1 completed
RMSE MCMC: 1473.706492529282


In [60]:
mu, sig, N = 1.1, 1, 100000

def q(x):
    return (1 / (math.sqrt(2 * math.pi * sig ** 2))) * (math.e ** (-((x - mu) ** 2) / (2 * sig ** 2)))

def MCMC(n):
    r = np.zeros(1)
    p = q(r[0])
    pts = []

    for i in range(N):
        rn = r + np.random.uniform(-1, 1)
        pn = q(rn[0])
        if pn >= p:
            p = pn
            r = rn
        else:
            u = np.random.rand()
            if u < pn / p:
                p = pn
                r = rn
        pts.append(r)

    pts = random.sample(pts, len(pts))
    pts = np.array(pts)

    return pts
maturnity = 1
volatility = 0.12
risk_free = 0.1
timestep = 1
steps = 22
delta_t = maturnity / steps

def MH(stock_name, data): 
    hist_data = yf.download(stock_name, start="2023-08-13", end="2023-09-04")
    hist_data = hist_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
    hist_data = hist_data['Close']
    hist_data = np.array(hist_data)
    stock_pred = []
    data = data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
    dt1 = data['Close']
    maturnity = 1
    volatility = 0.12
    risk_free = 0.1
    timestep = 1
    steps = 22
    delta_t = maturnity / steps
    i = 0
    stock_pred.append(dt1[-1])
    while timestep < steps:
        stock_price = stock_pred[-i]
        time_exp = maturnity - delta_t * timestep
        # Generate z_t using MCMC method
        pts = MCMC(N)
        stock_price = stock_price * math.exp(((risk_free - 0.5 * (
            math.pow(volatility, 2))) * delta_t + volatility * math.sqrt(delta_t) * pts[timestep + 5]))
        stock_pred.append(stock_price)
        i = i + 1
        timestep = timestep + 1

    rmse = math.sqrt(mean_squared_error(hist_data, stock_pred))
    print(f'RMSE MCMC: {rmse}')

    return stock_pred, rmse

def predict_tomorrows_price(stock_name, data):
    # Call the MH function to predict stock prices
    stock_pred, _ = MH(stock_name, data)

    # Extend the simulation by one more timestep
    timestep = 0
    timestep += 1
    stock_price = stock_pred[-1]
    pts = MCMC(N)
    tomorrow_price = stock_price * math.exp(((risk_free - 0.5 * (math.pow(volatility, 2))) * delta_t + volatility * math.sqrt(delta_t) * pts[timestep + 5]))

    print(f"Predicted price for tomorrow ({timestep} days from now): {tomorrow_price:.2f}")

    return tomorrow_price

# Usage example:
# Replace 'stock_name' and 'data' with your actual stock data
predicted_price = predict_tomorrows_price(stock_name, data)

[*********************100%***********************]  1 of 1 completed
RMSE MCMC: 1623.163016519842
Predicted price for tomorrow (1 days from now): 28641.24
